In [ ]:
''' 임시로 엑셀 변환하는 코드 '''
# import os
# import pandas as pd
# # 변환할 CSV 파일의 경로와 이름
# csv_file = "../dataset/본문_형태소분석/2018산림복지_본문_형태소분석(형용사).csv"

# # CSV 파일을 데이터프레임으로 읽기
# dataframe = pd.read_csv(csv_file, encoding='UTF-8')

# # XLSX 파일로 변환
# xlsx_file = os.path.splitext(csv_file)[0] + '.xlsx'
# dataframe.to_excel(xlsx_file, index=False)

In [1]:
import pandas as pd              # pandas 패키지 임포트
import numpy as np               # numpy 패키지 임포트
from konlpy.tag import Okt       # 한글 분석 패키지 konlpy 에서 Okt 형태소 분석기 임포트
from tqdm import tqdm     # for 반복문 진행상황 표시
from collections import Counter
# scikit-learn에서 제공하는 TF-IDF 생성 패키지 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()   # 코드 단순화를 위해 Okt()를 변수 okt에 할당

In [2]:
# 불용어 목록 만들기
# ※ 분석할 문장에서 조사를 뽑아 유일값만 남기면 됨.

df_josa = pd.read_csv("../dataset/2022/본문_형태소분석/2월/나무문화_2월_형태소분석(조사).csv", encoding='UTF-8')
josa_list = df_josa["조사"].tolist() # 이다, 보다는 지워야 함.
josa_list.remove("이다")
josa_list.remove("보다")
print(len(josa_list))

390


In [3]:
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_2월_형태소분석.txt", encoding='UTF-8')
df

,split_str,org_idx,morphs,pos_tag
0,집 앞에 바로 도서관이 있었는데 이사 오고 1년이 다 돼서야 알게 된 거 있지 구석...,0,"['집', '앞', '에', '바로', '도서관', '이', '있었는데', '이사'...","[('집', 'Noun'), ('앞', 'Noun'), ('에', 'Josa'), ..."
1,우웅 기준 오빠 목성의 위성 에우로파에는 관심없지만 이런 사랑이라면 훌훌 떠나도 좋...,0,"['우웅', '기준', '오빠', '목성', '의', '위성', '에우로파', '에...","[('우웅', 'Exclamation'), ('기준', 'Noun'), ('오빠',..."
2,우리반 임시 시간표인데 월요일 1교시 문학 임시라니까 언제까지 이러는 건지는 모르겠...,0,"['우리', '반', '임시', '시간표', '인데', '월요일', '1', '교시...","[('우리', 'Noun'), ('반', 'Noun'), ('임시', 'Noun')..."
3,약간 학교를 가고 싶기도 안 가고 싶기도 함,0,"['약간', '학교', '를', '가고', '싶기도', '안', '가고', '싶기도...","[('약간', 'Noun'), ('학교', 'Noun'), ('를', 'Josa')..."
4,물론 내 의사와는 상관없겠지만 껄껄 내 사랑 초코나무숲이 판매 종료 한다는 소식에 ...,0,"['물론', '내', '의사', '와는', '상관없겠지만', '껄껄', '내', '...","[('물론', 'Adverb'), ('내', 'Noun'), ('의사', 'Noun..."
...,...,...,...,...
700401,설날 작사 작곡 윤극영 노래 가사 쓰기 재생 12 좋아요 3 00 00 01 07 ...,13352,"['설날', '작사', '작곡', '윤극영', '노래', '가사', '쓰기', '재...","[('설날', 'Noun'), ('작사', 'Noun'), ('작곡', 'Noun'..."
700402,아동문학가이자 동요작곡가 그리고 아동문화운동가셨던 윤극영님은 푸른 하늘 은하수 하얀...,13352,"['아동문학', '가', '이자', '동요', '작곡가', '그리고', '아동', ...","[('아동문학', 'Noun'), ('가', 'Josa'), ('이자', 'Noun..."
700403,일제강점기 중 우리나라 가사를 붙인 노래들을 작곡하여 어린이들에게 민족에 대한 자긍...,13352,"['일제강점기', '중', '우리나라', '가사', '를', '붙인', '노래', ...","[('일제강점기', 'Noun'), ('중', 'Noun'), ('우리나라', 'N..."
700404,새해 복 많이 받으세요,13352,"['새해', '복', '많이', '받으세요']","[('새해', 'Noun'), ('복', 'Noun'), ('많이', 'Adverb..."


In [4]:
df['joined_morphs'] = df['morphs'].apply(lambda x: ' '.join(x.strip("]['").split("', '")))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs
0,집 앞에 바로 도서관이 있었는데 이사 오고 1년이 다 돼서야 알게 된 거 있지 구석...,0,"['집', '앞', '에', '바로', '도서관', '이', '있었는데', '이사'...","[('집', 'Noun'), ('앞', 'Noun'), ('에', 'Josa'), ...",집 앞 에 바로 도서관 이 있었는데 이사 오고 1년 이 다 돼 서야 알 게 된 거 ...
1,우웅 기준 오빠 목성의 위성 에우로파에는 관심없지만 이런 사랑이라면 훌훌 떠나도 좋...,0,"['우웅', '기준', '오빠', '목성', '의', '위성', '에우로파', '에...","[('우웅', 'Exclamation'), ('기준', 'Noun'), ('오빠',...",우웅 기준 오빠 목성 의 위성 에우로파 에는 관심 없지만 이런 사랑 이 라면 훌훌 ...
2,우리반 임시 시간표인데 월요일 1교시 문학 임시라니까 언제까지 이러는 건지는 모르겠...,0,"['우리', '반', '임시', '시간표', '인데', '월요일', '1', '교시...","[('우리', 'Noun'), ('반', 'Noun'), ('임시', 'Noun')...",우리 반 임시 시간표 인데 월요일 1 교시 문학 임시 라니까 언제 까지 이러는 건지...
3,약간 학교를 가고 싶기도 안 가고 싶기도 함,0,"['약간', '학교', '를', '가고', '싶기도', '안', '가고', '싶기도...","[('약간', 'Noun'), ('학교', 'Noun'), ('를', 'Josa')...",약간 학교 를 가고 싶기도 안 가고 싶기도 함
4,물론 내 의사와는 상관없겠지만 껄껄 내 사랑 초코나무숲이 판매 종료 한다는 소식에 ...,0,"['물론', '내', '의사', '와는', '상관없겠지만', '껄껄', '내', '...","[('물론', 'Adverb'), ('내', 'Noun'), ('의사', 'Noun...",물론 내 의사 와는 상관없겠지만 껄껄 내 사랑 초코 나무숲 이 판매 종료 한다는 소...
...,...,...,...,...,...
700401,설날 작사 작곡 윤극영 노래 가사 쓰기 재생 12 좋아요 3 00 00 01 07 ...,13352,"['설날', '작사', '작곡', '윤극영', '노래', '가사', '쓰기', '재...","[('설날', 'Noun'), ('작사', 'Noun'), ('작곡', 'Noun'...",설날 작사 작곡 윤극영 노래 가사 쓰기 재생 12 좋아요 3 00 00 01 07 ...
700402,아동문학가이자 동요작곡가 그리고 아동문화운동가셨던 윤극영님은 푸른 하늘 은하수 하얀...,13352,"['아동문학', '가', '이자', '동요', '작곡가', '그리고', '아동', ...","[('아동문학', 'Noun'), ('가', 'Josa'), ('이자', 'Noun...",아동문학 가 이자 동요 작곡가 그리고 아동 문화 운동 가셨던 윤극영 님 은 푸른 하...
700403,일제강점기 중 우리나라 가사를 붙인 노래들을 작곡하여 어린이들에게 민족에 대한 자긍...,13352,"['일제강점기', '중', '우리나라', '가사', '를', '붙인', '노래', ...","[('일제강점기', 'Noun'), ('중', 'Noun'), ('우리나라', 'N...",일제강점기 중 우리나라 가사 를 붙인 노래 들 을 작곡 하여 어린이 들 에게 민족 ...
700404,새해 복 많이 받으세요,13352,"['새해', '복', '많이', '받으세요']","[('새해', 'Noun'), ('복', 'Noun'), ('많이', 'Adverb...",새해 복 많이 받으세요


In [5]:
df['joined_sentences'] = df.groupby('org_idx')['joined_morphs'].transform(lambda x: ' '.join(x))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs,joined_sentences
0,집 앞에 바로 도서관이 있었는데 이사 오고 1년이 다 돼서야 알게 된 거 있지 구석...,0,"['집', '앞', '에', '바로', '도서관', '이', '있었는데', '이사'...","[('집', 'Noun'), ('앞', 'Noun'), ('에', 'Josa'), ...",집 앞 에 바로 도서관 이 있었는데 이사 오고 1년 이 다 돼 서야 알 게 된 거 ...,집 앞 에 바로 도서관 이 있었는데 이사 오고 1년 이 다 돼 서야 알 게 된 거 ...
1,우웅 기준 오빠 목성의 위성 에우로파에는 관심없지만 이런 사랑이라면 훌훌 떠나도 좋...,0,"['우웅', '기준', '오빠', '목성', '의', '위성', '에우로파', '에...","[('우웅', 'Exclamation'), ('기준', 'Noun'), ('오빠',...",우웅 기준 오빠 목성 의 위성 에우로파 에는 관심 없지만 이런 사랑 이 라면 훌훌 ...,집 앞 에 바로 도서관 이 있었는데 이사 오고 1년 이 다 돼 서야 알 게 된 거 ...
2,우리반 임시 시간표인데 월요일 1교시 문학 임시라니까 언제까지 이러는 건지는 모르겠...,0,"['우리', '반', '임시', '시간표', '인데', '월요일', '1', '교시...","[('우리', 'Noun'), ('반', 'Noun'), ('임시', 'Noun')...",우리 반 임시 시간표 인데 월요일 1 교시 문학 임시 라니까 언제 까지 이러는 건지...,집 앞 에 바로 도서관 이 있었는데 이사 오고 1년 이 다 돼 서야 알 게 된 거 ...
3,약간 학교를 가고 싶기도 안 가고 싶기도 함,0,"['약간', '학교', '를', '가고', '싶기도', '안', '가고', '싶기도...","[('약간', 'Noun'), ('학교', 'Noun'), ('를', 'Josa')...",약간 학교 를 가고 싶기도 안 가고 싶기도 함,집 앞 에 바로 도서관 이 있었는데 이사 오고 1년 이 다 돼 서야 알 게 된 거 ...
4,물론 내 의사와는 상관없겠지만 껄껄 내 사랑 초코나무숲이 판매 종료 한다는 소식에 ...,0,"['물론', '내', '의사', '와는', '상관없겠지만', '껄껄', '내', '...","[('물론', 'Adverb'), ('내', 'Noun'), ('의사', 'Noun...",물론 내 의사 와는 상관없겠지만 껄껄 내 사랑 초코 나무숲 이 판매 종료 한다는 소...,집 앞 에 바로 도서관 이 있었는데 이사 오고 1년 이 다 돼 서야 알 게 된 거 ...
...,...,...,...,...,...,...
700401,설날 작사 작곡 윤극영 노래 가사 쓰기 재생 12 좋아요 3 00 00 01 07 ...,13352,"['설날', '작사', '작곡', '윤극영', '노래', '가사', '쓰기', '재...","[('설날', 'Noun'), ('작사', 'Noun'), ('작곡', 'Noun'...",설날 작사 작곡 윤극영 노래 가사 쓰기 재생 12 좋아요 3 00 00 01 07 ...,설날 작사 작곡 윤극영 노래 가사 쓰기 재생 12 좋아요 3 00 00 01 07 ...
700402,아동문학가이자 동요작곡가 그리고 아동문화운동가셨던 윤극영님은 푸른 하늘 은하수 하얀...,13352,"['아동문학', '가', '이자', '동요', '작곡가', '그리고', '아동', ...","[('아동문학', 'Noun'), ('가', 'Josa'), ('이자', 'Noun...",아동문학 가 이자 동요 작곡가 그리고 아동 문화 운동 가셨던 윤극영 님 은 푸른 하...,설날 작사 작곡 윤극영 노래 가사 쓰기 재생 12 좋아요 3 00 00 01 07 ...
700403,일제강점기 중 우리나라 가사를 붙인 노래들을 작곡하여 어린이들에게 민족에 대한 자긍...,13352,"['일제강점기', '중', '우리나라', '가사', '를', '붙인', '노래', ...","[('일제강점기', 'Noun'), ('중', 'Noun'), ('우리나라', 'N...",일제강점기 중 우리나라 가사 를 붙인 노래 들 을 작곡 하여 어린이 들 에게 민족 ...,설날 작사 작곡 윤극영 노래 가사 쓰기 재생 12 좋아요 3 00 00 01 07 ...
700404,새해 복 많이 받으세요,13352,"['새해', '복', '많이', '받으세요']","[('새해', 'Noun'), ('복', 'Noun'), ('많이', 'Adverb...",새해 복 많이 받으세요,설날 작사 작곡 윤극영 노래 가사 쓰기 재생 12 좋아요 3 00 00 01 07 ...


In [6]:
df_tf = df[['org_idx', 'joined_sentences']].drop_duplicates()
df_tf.reset_index(drop=True, inplace=True)
df_tf

,org_idx,joined_sentences
0,0,집 앞 에 바로 도서관 이 있었는데 이사 오고 1년 이 다 돼 서야 알 게 된 거 ...
1,1,안녕하세요 꼼꼼 여사 에요 오늘 은 김나륜 작품 초 대전 을 다녀왔어요 저 는 그림...
2,2,다큐멘터리 미스 아메리카 나 줄거리 테일러 스위프트 의 삶 유튜브 에서 추천 하길래...
3,3,대한민국 공간 문화 대상 대상 황매산 군 립 공원 휴게소 철쭉 과 억새 사이 관광객...
4,4,046 단상 건축 과 도시 파일럿 도시 는 건축 의 확장 형 이다 적어도 도시 는 ...
...,...,...
13348,13348,31일 밤 방송 된 KBS 2 TV 자본주의 학교 에는 도 신해철 와이프 윤원희 씨...
13349,13349,2022 임인년 설날 입니다 명절 연휴 다 들 잘 보내고 계신지요 저 는 차례 드릴...
13350,13350,신세계 백화점 타임스퀘어 점 문화센터 꽃꽂이 수강 후기 50 m NAVER Corp...
13351,13351,전북 가볼만 한 곳 고창 고인돌 유적지 운곡 습지 고창읍성 등 고창 은 서해 바다 ...


In [7]:
# TF-IDF 생성
""" CountVectorizer로 TF-IDF를 만들어서 변환하는 방법도 있지만 """
""" 처음부터 Tfidf를 구해주는 함수(TfidfVectorizer)가 있기 때문에 이를 사용할 것임. """

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# 데이터 프레임에서 분석할 문서를 가져와서 쓸 경우 꼭 리스트로 변환해서 담아줘야 함.
corpus = df_tf['joined_sentences'].to_list()

In [8]:
# stop_words에는 위에서 조사만 추출해서 만든 불용어 리스트 사용
tfidf = TfidfVectorizer(stop_words = josa_list, token_pattern=r'\w{1,}')
# tfidf_v = TfidfVectorizer(stop_words = hyeong_josa2).fit(corpus)
tdm = tfidf.fit_transform(corpus)

In [9]:
tdm.shape

(13353, 381732)

In [10]:
# 만든 TF-IDF를 데이터 프레임에 담기 위해 우선 numpy.array()로 변환함
tfidf_array = tdm.toarray()
tfidf_array

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.01155767, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.0083323 , 0.01565925, 0.02470766, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04987606, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [11]:
# 변환한 array를 다시 pd.Dataframe으로 바꿈
tfidf_DF = pd.DataFrame(tfidf_array)
tfidf_DF

,0,1,2,3,4,5,6,7,8,9,...,381722,381723,381724,381725,381726,381727,381728,381729,381730,381731
0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13348,0.011734,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13349,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13350,0.011558,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13351,0.008332,0.015659,0.024708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# 이 데이터프레임의 컬럼명은 숫자로 되어 있음. 이를 feature_names로 바꿔줘야 함.
featurenames = tfidf.get_feature_names_out()
tfidf_DF.columns = featurenames

In [13]:
tfidf_DF

,0,00,000,0001,00010,000111,0001190422,00014,00016,0001645260,...,힛데겔,힛데켈,힛송,힛트,힜,힝,힝구,힝힝,힠,힣
0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13348,0.011734,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13349,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13350,0.011558,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13351,0.008332,0.015659,0.024708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
tfidf_DF.to_csv("../dataset/2022/본문_TF-IDF/2월/나무문화_2월_TF-IDF행렬(원본).csv", index=False, encoding='UTF-8')

In [14]:
import re
# 데이터프레임에서 한글로만 이루어진 column 필터링을 위한 정규표현식 패턴
pattern = r'^[ㄱ-ㅎㅏ-ㅣ가-힣\s]+$'

# 한글로만 이루어진 column으로 된 데이터프레임 생성
korean_df = tfidf_DF[[column for column in tfidf_DF.columns if re.match(pattern, column)]]

print(korean_df)

         ㄱ   ㄱㄱ  ㄱㄱㄱ  ㄱㄱㅎ  ㄱㄴㄷ  ㄱㄴㄷㄹ   ㄱㅅ  ㄱㅅㄱㅅ  ㄱㅇㄱ  ㄱㅇㄷ  ...  힛데겔  힛데켈   힛송  \
0      0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0  0.0   
1      0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0  0.0   
2      0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0  0.0   
3      0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0  0.0   
4      0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0  0.0   
...    ...  ...  ...  ...  ...   ...  ...   ...  ...  ...  ...  ...  ...  ...   
13348  0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0  0.0   
13349  0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0  0.0   
13350  0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0  0.0   
13351  0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0  0.0   
13352  0.0  0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0  0.0   

        힛트    힜    힝   힝구  

In [ ]:
# 한글로만 이루어진 TF-IDF 행렬도 txt로 저장해줌.
korean_df.to_csv("../dataset/2022/본문_TF-IDF/2월/나무문화_2월_TF-IDF행렬(한글만).csv", index=False, encoding='UTF-8')

In [15]:
# 각 단어에 대한 TF-IDF의 합
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_count

,단어,tf-idf
0,0,66.128836
1,00,157.579722
2,000,23.715986
3,0001,0.611613
4,00010,0.010911
...,...,...
381727,힝,0.997458
381728,힝구,0.238519
381729,힝힝,0.141211
381730,힠,0.058112


In [16]:
# 단어 - TF-IDF 파일로 저장.
word_count.to_csv("../dataset/2022/본문_TF-IDF/2월/나무문화_2월_TF-IDF행렬합.txt", index=False, encoding='UTF-8')